In [4]:
import pandas as pd
import pandas_profiling as pdp
import lightgbm as lgb
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
import optuna

In [5]:
train = pd.read_csv('../processed_data/train_v10.csv')
test = pd.read_csv('../processed_data/test_v10.csv')

In [6]:
use_col = train.columns

un_use_col = ['id','y','log_y','high_price_flag','location', 'access', 'layout', 'age', 'direction', 'area','floor', 'bath_toilet', 'kitchen',
             'broadcast_com', 'facilities','parking', 'enviroment', 'structure', 'contract_period',
             'walk_time','23ku',
            #  'area_num_countall','floor_countall','room_num_countall','facilities_countall','age_countall','area_num_countall',
            ]
use_col = [c for c in use_col if c not in un_use_col]

In [7]:
X, y = train.loc[:,use_col], train.loc[:,'high_price_flag']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [9]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [10]:
lgbm_params = {
        # 多値分類問題
        'objective': 'binary',
        'metric': 'accuracy',
        'stratified': True
    }

# 上記のパラメータでモデルを学習する
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

# テストデータを予測する
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [11]:
pred_label = [1 if i>0.5 else 0 for i in y_pred]

In [12]:
accuracy = accuracy_score(y_test,pred_label)

In [13]:
accuracy

0.9956572396991844

In [14]:
pd.DataFrame({
    'gt': y_test,
    'pred': pred_label
})

,gt,pred
27300,0,0
2953,0,0
8326,0,0
2921,0,0
19929,0,0
...,...,...
17358,0,0
4024,0,0
9721,0,0
12102,0,0


In [15]:
def objective(trial):

    learning_rate = trial.suggest_uniform('learning_rate', 0, 1.0)
    num_leaves = trial.suggest_int('num_leaves', 10, 2**8)
    max_depth = trial.suggest_int('max_depth', 3, 8)
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 2, 1000)

    lgbm_params = {
        'task': 'train',
        "metric": 'binary',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        "learning_rate": learning_rate,
        "num_leaves": num_leaves,
        "max_depth": max_depth,
        "n_jobs": 1,
        'verbose': -1,
        "seed": 0
    }

    cv_results = lgb.cv(lgbm_params, lgb_train, nfold=3, stratified=True)
    score = np.array(cv_results['binary_logloss-mean']).mean()
    # accuracy = accuracy_score(y_test,pred_label)
    return score

In [16]:
study = optuna.create_study()
study.optimize(objective,n_trials=10)

[I 2019-10-06 18:36:56,709] Finished trial#0 resulted in value: 0.22373274494338702. Current best value is 0.22373274494338702 with parameters: {'learning_rate': 0.2055786152963398, 'num_leaves': 170, 'max_depth': 4, 'min_data_in_leaf': 884}.
[I 2019-10-06 18:36:57,394] Finished trial#1 resulted in value: 0.7321009825720379. Current best value is 0.22373274494338702 with parameters: {'learning_rate': 0.2055786152963398, 'num_leaves': 170, 'max_depth': 4, 'min_data_in_leaf': 884}.
[I 2019-10-06 18:36:58,295] Finished trial#2 resulted in value: 0.012977225471946125. Current best value is 0.012977225471946125 with parameters: {'learning_rate': 0.06323787278004289, 'num_leaves': 50, 'max_depth': 4, 'min_data_in_leaf': 284}.
[I 2019-10-06 18:36:59,003] Finished trial#3 resulted in value: 0.415190047592709. Current best value is 0.012977225471946125 with parameters: {'learning_rate': 0.06323787278004289, 'num_leaves': 50, 'max_depth': 4, 'min_data_in_leaf': 284}.
[I 2019-10-06 18:36:59,622] 

In [91]:
print('best trial: ', study.best_trial)
print('---------------------------------')
print('best_params: ', study.best_params)

best trial:  FrozenTrial(number=8, state=<TrialState.COMPLETE: 1>, value=0.038809788678887076, datetime_start=datetime.datetime(2019, 10, 5, 16, 33, 59, 267502), datetime_complete=datetime.datetime(2019, 10, 5, 16, 34, 0, 632326), params={'learning_rate': 0.14264489475165743, 'num_leaves': 100, 'max_depth': 4, 'min_data_in_leaf': 337}, distributions={'learning_rate': UniformDistribution(low=0, high=1.0), 'num_leaves': IntUniformDistribution(low=10, high=256), 'max_depth': IntUniformDistribution(low=3, high=8), 'min_data_in_leaf': IntUniformDistribution(low=2, high=1000)}, user_attrs={}, system_attrs={'_number': 8}, intermediate_values={}, params_in_internal_repr={'learning_rate': 0.14264489475165743, 'num_leaves': 100, 'max_depth': 4, 'min_data_in_leaf': 337}, trial_id=8)
---------------------------------
best_params:  {'learning_rate': 0.14264489475165743, 'num_leaves': 100, 'max_depth': 4, 'min_data_in_leaf': 337}


In [17]:
mdl = lgb.train(study.best_params, lgb_train)

In [18]:
mdl.predict(X_train,metric='accuracy')

array([-7.32105455e-05, -5.25067118e-03, -8.12912031e-05, ...,
       -5.89339351e-04, -6.46221322e-05, -6.46221322e-05])

In [2]:
p = 'submit/1234_submit.csv'

In [3]:
import os

In [7]:
a,b = os.path.splitext(p)

In [8]:
a

'submit/1234_submit'

In [9]:
b

'.csv'

In [11]:
a+'low'+b

'submit/1234_submitlow.csv'